# Simplex-Algorithmus

Implementation des Algorithmus' wie er im Skript steht. Achtung: Dies ist eine konzeptionelle Version des Algorithmus', die für die Praxis ungeeignet ist. 

In [1]:
import numpy as np

In [11]:
A=np.array(np.arange(12).reshape(-1,4))
b=np.array([27,12,42]).reshape(-1,1)
A,b

(array([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]),
 array([[27],
        [12],
        [42]]))

In [22]:
add_nonneg(A,b)

(array([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [-1., -0., -0., -0.],
        [-0., -1., -0., -0.],
        [-0., -0., -1., -0.],
        [-0., -0., -0., -1.]]),
 array([[27.],
        [12.],
        [42.],
        [ 0.],
        [ 0.],
        [ 0.]]))

In [30]:
def add_nonneg(A,b):
    m,n=A.shape
    b=np.vstack([b,np.zeros(n).reshape(-1,1)])
    A=np.vstack([A,-np.eye(n)])
    return A,b

def twisted_cube(n=3):
    c=np.array([2**i for i in range(n-1,-1,-1)]).reshape(-1,1)
    b=np.array([5**i for i in range(1,n+1)]).reshape(-1,1)
    A=2*np.array([[int(2**(i-j)) for j in range(n)] for i in range(n)])
    for i in range(n):
        A[i,i]=1
    A,b=add_nonneg(A,b)
    R=np.array([[i+n,i] for i in range(n)]).flatten()
    A=A[R].copy()
    b=b[R].copy()
    x0=np.zeros(n).reshape(-1,1)
    return A,b,c,x0



In [15]:
def unit_vec(i,n):
    vec=np.zeros(n)
    vec[i]=1
    return vec.reshape(-1,1)

class State:
    def __init__(self,x=None,J=None,y_J=None,i=None,jstar=None,w=None,opt_status="running",lmbda=None,new_x=None,objective=None):
        self.x=x.flatten()
        self.J=J.copy() if J is not None else None
        self.y_J=y_J.flatten() if y_J is not None else None 
        self.i=i
        self.jstar=jstar
        self.w=w.flatten() if w is not None else None
        self.opt_status=opt_status
        self.lmbda=lmbda
        self.new_x=new_x.flatten() if new_x is not None else None
        self.objective=objective
    
    def __repr__(self):
        repr=""
        for key,value in vars(self).items():
            if value is not None:
                repr+="{}: {}\n".format(key,value)
        return repr+"\n"

class Simplex:
    def __init__(self,A,b,c,x,max_iter=100,eps=1E-8):
        self.A=A
        self.b=b
        self.c=c
        self.x=x
        self.eps=eps
        self.record=[]
        self.done=False
        self.max_iter=max_iter
        self.check_start_vertex()
        self.init_representation()
        self.run()
        
    def check_start_vertex(self):
        if not np.all(self.A@self.x-self.b <=self.eps):
            raise Exception("start vertex not feasible!")
        
    def current_objective(self):
        return sum(self.x.T@self.c)
        
    def run(self):
        for _ in range(self.max_iter):
            if self.done:
                break
            self.simplex_step()
        
    def init_representation(self):
        self.J=[j for j,a in enumerate(self.A) if abs(a@self.x-self.b[j])<self.eps]
        self.reduce_index_set() # make sure that |J|=n
        
    def reduce_by_one(self):
        m,n=self.A.shape
        for j in range(len(self.J)):
            JJ=self.J.copy()
            del JJ[j]
            if np.linalg.matrix_rank(self.A[JJ])>=n:
                break
        del self.J[j]
        
    def reduce_index_set(self):
        m,n=self.A.shape
        while len(self.J)>n:
            self.reduce_by_one()
        
    def simplex_step(self):
        A_J=self.A[self.J]
        self.y_J=np.linalg.solve(A_J.T,self.c)
        obj=self.current_objective()
        state=State(x=self.x,J=self.J,y_J=self.y_J,objective=obj)
        self.record.append(state)
        if np.all(self.y_J>=0):
            self.done=True
            self.opt_status="OPT found"
            state.opt_status="OPT found"
            return
        for rel_i,yy in enumerate(self.y_J):
            if yy<-self.eps:
                break
        state.i=self.J[rel_i]
        w=np.linalg.solve(A_J,-unit_vec(rel_i,len(self.J)))
        state.w=w.flatten()
        if np.all(A@w<=self.eps):
            self.done=True
            self.opt_status="unbounded"
            state.opt_status="unbounded"
            return
        lambdas=[(sum((self.b[j]-sum(self.A[j]@self.x))/sum(self.A[j]@w)),j) for j in range(len(self.A)) if sum(self.A[j]@w)>self.eps]
        lmbda,jstar=min(lambdas,key=lambda l:l[0])
        state.jstar=jstar
        state.lmbda=lmbda
        self.J[rel_i]=jstar
        self.J.sort()
        self.x=np.linalg.solve(self.A[self.J],self.b[self.J])
        state.new_x=self.x.flatten()


In [167]:

Simplex(np.eye(3),-np.ones(3).reshape(-1,1),None,np.zeros(3).reshape(-1,1))

Exception: start vertex not feasible!

In [31]:
A,b,c,x0=twisted_cube(3)
simplex=Simplex(A,b,c,x0)
len(simplex.record)

8

In [34]:
A,b,c,x0=twisted_cube(6)
simplex=Simplex(A,b,c,x0)
len(simplex.record)

64

In [35]:
A

array([[-1., -0., -0., -0., -0., -0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.],
       [-0., -1., -0., -0., -0., -0.],
       [ 4.,  1.,  0.,  0.,  0.,  0.],
       [-0., -0., -1., -0., -0., -0.],
       [ 8.,  4.,  1.,  0.,  0.,  0.],
       [-0., -0., -0., -1., -0., -0.],
       [16.,  8.,  4.,  1.,  0.,  0.],
       [-0., -0., -0., -0., -1., -0.],
       [32., 16.,  8.,  4.,  1.,  0.],
       [-0., -0., -0., -0., -0., -1.],
       [64., 32., 16.,  8.,  4.,  1.]])

In [36]:
c.flatten()

array([32, 16,  8,  4,  2,  1])

In [37]:
A,b,c,x0=twisted_cube(3)
A

array([[-1., -0., -0.],
       [ 1.,  0.,  0.],
       [-0., -1., -0.],
       [ 4.,  1.,  0.],
       [-0., -0., -1.],
       [ 8.,  4.,  1.]])

In [29]:
simplex=Simplex(AA,bb,c,x0)
simplex.record

[x: [0. 0. 0.]
 J: [0, 2, 4]
 y_J: [-4. -2. -1.]
 i: 0
 jstar: 1
 w: [ 1. -0. -0.]
 opt_status: running
 lmbda: 5.0
 new_x: [ 5. -0. -0.]
 objective: [0.]
 ,
 x: [ 5. -0. -0.]
 J: [1, 2, 4]
 y_J: [ 4. -2. -1.]
 i: 2
 jstar: 3
 w: [ 0.  1. -0.]
 opt_status: running
 lmbda: 5.0
 new_x: [ 5.  5. -0.]
 objective: [20.]
 ,
 x: [ 5.  5. -0.]
 J: [1, 3, 4]
 y_J: [-4.  2. -1.]
 i: 1
 jstar: 0
 w: [-1.  4. -0.]
 opt_status: running
 lmbda: 5.0
 new_x: [ 0. 25. -0.]
 objective: [30.]
 ,
 x: [ 0. 25. -0.]
 J: [0, 3, 4]
 y_J: [ 4.  2. -1.]
 i: 4
 jstar: 5
 w: [-0.  0.  1.]
 opt_status: running
 lmbda: 25.0
 new_x: [ 0. 25. 25.]
 objective: [50.]
 ,
 x: [ 0. 25. 25.]
 J: [0, 3, 5]
 y_J: [-4. -2.  1.]
 i: 0
 jstar: 1
 w: [ 1. -4.  8.]
 opt_status: running
 lmbda: 5.0
 new_x: [ 5.  5. 65.]
 objective: [75.]
 ,
 x: [ 5.  5. 65.]
 J: [1, 3, 5]
 y_J: [ 4. -2.  1.]
 i: 3
 jstar: 2
 w: [ 0. -1.  4.]
 opt_status: running
 lmbda: 5.0
 new_x: [ 5. -0. 85.]
 objective: [95.]
 ,
 x: [ 5. -0. 85.]
 J: [1, 2, 5]

In [88]:
simplex.lambdas

[array([5.]), array([6.25]), array([15.625])]

In [89]:
np.argmin(simplex.lambdas)

0

In [54]:
simplex.simplex_step()
simplex.y_J

array([[-4.],
       [-2.],
       [-1.]])

In [49]:
B=simplex.A[simplex.J].T
B

array([[-1., -0., -0.],
       [-0., -1., -0.],
       [-0., -0., -1.]])

In [32]:
[j for j,a in enumerate(A)]

[0, 1, 2, 3, 4, 5]

In [34]:
a=A[2]
a

array([8., 4., 1.])

In [39]:
A_J=A[simplex.J]
A_J

array([[-1., -0., -0.],
       [-0., -1., -0.],
       [-0., -0., -1.]])

In [40]:
A_Jinv=np.linalg.inv(A_J)
A_Jinv

array([[-1., -0., -0.],
       [-0., -1., -0.],
       [-0., -0., -1.]])